### Lazy Propagation
- Segment Tree에서 노드의 업데이트를 미루다가, 쿼리를 받으면 그때 업데이트를 하는 방식
- 각 노드에 대응하는 lazy 배열을 두어, 업데이트를 미룬다.
  - 해당 노드가 관장하는 구간 전체에 대해서 어떤 연산을 미루고 있는지에 대한 정보가 들어있다.
    - 구간합 세그의 경우 얼마를 더할 것인지, xor 세그의 경우 부호를 바꿀지의 여부 등
- 쿼리를 받으면, 해당 노드의 lazy를 업데이트하고, 자식 노드에게 propagate한다.
- 길이 $N$인 구간 $[a, b]$의 모든 인덱스에 연산을 적용하는 쿼리 $Q$ 의 구현을 Naive하게 적용한다면 그 구간의 길이 $b-a = K$ 라고 했을 때 $O(K \log N)$ 이 걸린다.\
하지만 Lazy Propagation을 적용하면 $O(\log N)$ 으로 줄일 수 있다.

#### Propagate
- 완전히 포함되면 lazy만 매기고, propagate한다

### 구현(재귀)
- query의 내부를 수정하기 쉬운 템플릿
- 코드 주석은 10999(구간 합 구하기 2) 참고

In [ ]:
class LazySegmentTree:
  def __init__(self, data):
    self.size = 1 << (len(data) - 1).bit_length()
    self.L = [0] * (2 * self.size)
    self.lazy = [0] * (2 * self.size)
    self.L[self.size:self.size + len(data)] = data
    for i in reversed(range(self.size)):
      self.L[i] = self.L[i*2] + self.L[i*2+1]

  def push(self, i, s, e) :
    if not self.lazy[i] : return

    self.L[i] += (e - s) * self.lazy[i]
    if i < self.size :
      self.lazy[i*2] += self.lazy[i]
      self.lazy[i*2+1] += self.lazy[i]
    self.lazy[i] = 0

  def query(self, l, r):
    return self._query(l, r, 1, 0, self.size)

  def _query(self, l, r, i, nl, nr) :
    self.push(i, nl, nr)
    
    if r <= nl or nr <= l : return 0
    if l <= nl and nr <= r : return self.L[i]
    mid = (nl + nr) // 2
    return self._query(l, r, i*2, nl, mid) + self._query(l, r, i*2+1, mid, nr)
  
  def update(self, l, r, x):
    self._update(l, r, x, 1, 0, self.size)

  def _update(self, l, r, x, i, nl, nr) :
    self.push(i, nl, nr)

    if r <= nl or nr <= l : return
    if l <= nl and nr <= r :
      self.lazy[i] += x
      self.push(i, nl, nr)
      return
  
    mid = (nl + nr) // 2
    self._update(l, r, x, i*2, nl, mid)
    self._update(l, r, x, i*2+1, mid, nr)
    self.L[i] = self.L[i*2] + self.L[i*2+1]

### iterative 구현
- [s, e)를 정말 하고싶었는데, 우선 [s, e]으로 구현돼있음에 주의
- update와 query 이전에 구간에 대한 propagate가 진행된다.
- build는 조상 방향, propgate는 자식 방향에게 쿼리에 대한 정보를 알린다.
- 쿼리할 때 들어가는 연산 `x`는 `LazyNode`과 자료구조가 같다.
- f(a: `Node`, b: `Node`) -> `Node`
  - 노드와 lazy끼리의 연산
  - 값을 구하기 위해 필요한 연산
- fl(v: `Node`, x: `LazyNode`, cnt: `int`) -> `Node`
  - 노드 `v`에 대해 연산 `x` 서브노드의 개수 `cnt`번 반복한 것과 같은 값을 적용시킨다.
  - x가 cnt회 반복될 때의 특성을 잘 이해해야 한다.
- fll(lv: `LazyNode`, x: `LazyNode`) -> `LazyNode`
  - 레이지 노드 `lv` 에 대해 연산 `x`를 적용시킨다.
  - 리프 노드가 아닌 lazy끼리의 연산
  - 연산 순서는 `lv`보다 `x`가 더 앞선다. 어떤 수를 대입하는 연산이라면 `x`로 대입된다고 보면 된다.
- `for i in range(N) : lst._apply(i)` seg 값을 디버그해야 한다면 이 코드를 수행시킨 후에 해야한다.

#### f, fl, fll 템플릿
- 구간합일 경우
  - f = add
  - fl = lambda i, x, cnt: i+x*cnt
  - fll = add
- 구간 max일 경우 
  - f = max
  - fl = lambda i, x, cnt: i+x
  - fll = add

In [ ]:
from typing import Callable as F
class LST:
  def __init__(self, L, f: F[[int, int]], fl: F[[int, int, int]], fll: F[[int, int]], default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(L)
    self.seg = [default] * self.len + L
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _push(self, i, x) :
    self.seg[i] = self.fl(self.seg[i], x, self.cnt[i])
    if i < self.len :
      self.lazy[i] = self.fll(self.lazy[i], x)

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _build(self, i) :
    while i :
      if i < self.len :
        self._apply(i)
        self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      i >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.seg[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.seg[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)